Join the data from Part 1 with the data from Part 2 to create a new dataframe.

In [ ]:
FS_df_with_station_id = pd.merge(poi_for_all_stations, station_to_poi_table, how='inner', on='fsq_id')
joined_data = pd.merge(FS_df_with_station_id, station_stats, how='inner', left_on='station_id', right_on= "id" )
joined_data.rename(columns={"name_x": "name","name_y": "street_names"}, inplace=True)
joined_data.drop(columns=['id'],inplace=True)

Provide a visualization that you used as part of your EDA process. Explain the initial pattern or relationship you discoved through this visualization. 

In [ ]:
import seaborn as sns

In [ ]:
each_POI_count=joined_data.groupby(['fsq_id']).agg({'fsq_id': 'count', 'rating':'mean', 'popularity':'mean'}).reset_index(drop = True)
grouped_by_POI_count = each_POI_count.groupby(['fsq_id'])[['rating','popularity']].mean().reset_index()

bins = [0,5,10,15,20,25,30,35,40,45]
grouped_by_POI_count = grouped_by_POI_count.groupby(pd.cut(grouped_by_POI_count['fsq_id'], bins=bins))[['rating','popularity']].mean().reset_index()

# one visualization
relation_between_popularity_and_num_of_bike_stations = sns.barplot(data=grouped_by_POI_count,x = "fsq_id", y="popularity")
relation_between_popularity_and_num_of_bike_stations.set(xlabel='Number of bike stations in a 1km radius',
       ylabel='Popularity',
       title='Number of bike stations X Popularity')

# another visualization
category_stats = pd.merge(cat_details_table, cat_table, how='inner', left_on='cat_id', right_on= "Category_id" ).drop_duplicates(subset = ["fsq_id","Category_id"], ignore_index = True)
category_count = category_stats["cat_id"].value_counts()
category_count.plot(kind='pie')

# Database

Put all your results in an SQLite3 database (remember, SQLite stores its databases as files in your local machine - make sure to create your database in your project's data/ directory!)

In [ ]:
import sqlite3
from sqlite3 import Error

In [ ]:
def create_connection(path):
    connection = None
    try:
        connection = sqlite3.connect(path)
        print("Connection to SQLite DB successful")
    except Error as e:
        print(f"The error '{e}' occurred")

    return connection

connection = create_connection("Hamilton_DB.sqlite")

poi_for_all_stations.drop(columns=['categories'],inplace=True)

cat_details_table.to_sql(name = "cat_details", con= connection,if_exists="replace",index=False)
station_to_poi_table.to_sql(name = "stations_POI", con= connection,if_exists="replace",index=False)
station_stats.to_sql(name = "all_stations", con= connection,if_exists="replace",index=False)
cat_table.to_sql(name = "POI_Category", con= connection,if_exists="replace",index=False)
poi_for_all_stations.to_sql(name = "all_POI", con= connection,if_exists="replace",index=False)

Look at the data before and after the join to validate your data.